In [1]:
import os
import requests
from ollama import chat
from ollama import ChatResponse
from firecrawl import FirecrawlApp
from pydantic import BaseModel
from typing import Optional
from dotenv import load_dotenv
from IPython.display import display, Markdown
from requests.exceptions import HTTPError
load_dotenv()

SERP_API_KEY = os.getenv("SERP_API_KEY")
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY")
MODEL = os.getenv("MODEL")

In [2]:
def fetch_serp(q: str):
   url = 'https://serpapi.com/search'
   params = {
        'api_key': SERP_API_KEY,
        'engine': 'google',
        'q': q,
        'google_domain': 'google.com',
        'hl': 'en'
    }
   response = requests.get(url, params=params)
   if response.status_code == 200:
        json_response = response.json()
        organic = json_response['organic_results']
        formatted_results = []
        for result in organic:
            title = result['title']
            link = result['link']
            description = result['snippet']
            resource_object = {
                'title': title,
                'link': link,
                'description': description
            }
            formatted_results.append(resource_object)
        return formatted_results

In [3]:
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
def retrieve_context(url:str):
    context = app.scrape_url(url, params={'formats':['markdown']})
    return context

In [4]:
class SearchQueries(BaseModel):
    queries: list[str] 

user_prompt = input("Topic of Research: ")
research_objective = input("Research objective: ")
research_width = input("Width of Research (1-5): ")

def generate_queries(prompt:str, objective:str, width:str):
    generated_queries: ChatResponse = chat(model=MODEL, messages=[
    {
        'role': 'user',
        'content': f"Given the user's prompt, generate {width} search queries (to be used as Google Search queries) that will get you started with performing deep research on the topic in hand. It is important that the queries you generate are unique and have ZERO overlap with each other. The number of search queries you are asked to generate defines the width of the user's research. Do not generate anything else apart from the {width} queries required. Here's the user's research title:{prompt}. And here's the research objective: {objective}. Your thought process while generating these queries should consider the main objective of the research, and you should figure out how to progressively.",
    },
    ],
    format=SearchQueries.model_json_schema())
    formatted_queries = SearchQueries.model_validate_json(generated_queries.message.content)
    return formatted_queries

In [5]:
search_results = []
current_queries = generate_queries(user_prompt, research_objective, research_width)
for q in current_queries.queries:
    results = fetch_serp(q)
    search_results.extend(results)
%store search_results
print(search_results)

Stored 'search_results' (list)
[{'title': 'Australia Aviation Market Size & Share Analysis', 'link': 'https://www.mordorintelligence.com/industry-reports/australia-aviation-market', 'description': 'The Australia Aviation Market is expected to reach USD 4.14 billion in 2025 and grow at a CAGR of 1.53% to reach USD 4.46 billion by 2030.'}, {'title': 'Domestic Airlines in Australia - Market Research Report ...', 'link': 'https://www.ibisworld.com/australia/industry/domestic-airlines/472/', 'description': 'Market size and recent performance (2014-2029). Industry revenue has grown at a CAGR of 4.9 % over the past five years, to reach an estimated $17.2bn in 2024.'}, {'title': 'Flights - Australia | Statista Market Forecast', 'link': 'https://www.statista.com/outlook/mmo/shared-mobility/flights/australia', 'description': 'The Flights market in in Australia is projected to grow by 2.66% (2025-2029) resulting in a market volume of US$12.72bn in 2029.'}, {'title': 'Airline competition in Austra

/Users/sanandh/Desktop/Projects/Deep-Research-Python/venv/lib/python3.12/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/search_results requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [6]:
class ResultItem(BaseModel):
    title: str
    link: str

class RelevantQueries(BaseModel):
    r_queries: list[ResultItem]

references = []

def evaluate_relevance(prompt:str, objective:str, width:int, serp_results):
    relevant_queries: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content': f"You are an expert data extractor. You are provided with a list of search results for a specific research query. This will be a list of objects containing the title, description and the link to an article/webpage. Use the title and description to verify if a particular search result is indeed relevant to the query. Return the titles AND links that are relevant to the research objective in the specified format. Here is the list of search results: {serp_results}. Here's the user's topic of research: {prompt}. The objective of the research is as follows: {objective}. You must return {width} results from the list that has been provided to you"
        }
    ],format=RelevantQueries.model_json_schema())
    formatted_relevant_queries = RelevantQueries.model_validate_json(relevant_queries.message.content)
    references.extend(formatted_relevant_queries.r_queries)
    return formatted_relevant_queries, references

In [7]:
research_context = []
evaluated_queries, _ = evaluate_relevance(user_prompt, research_objective, research_width, search_results)

In [8]:
evaluated_queries.r_queries

[ResultItem(title="Qantas' brand health plummets; Virgin Australia sees gains in consumer sentiment", link='https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/'),
 ResultItem(title='Domestic airline competition in Australia | May 2024 report', link='https://www.accc.gov.au/system/files/domestic-airline-competition-in-australia-may-2024-report.pdf'),
 ResultItem(title='Insights on Australian travel behaviours', link='https://www.thinkwithgoogle.com/intl/en-apac/consumer-insights/consumer-trends/insights-aunz-traveler-behaviors/')]

In [9]:
for item in evaluated_queries.r_queries:
    print(f"Retrieving research context for the resource: {item.title}")
    try:
        c = retrieve_context(item.link)
        research_context.append(c.get("markdown", ""))
    except HTTPError as e:
        print(f"Failed to retrieve content from {item.title}: {str(e)}")
        references.remove(item)
        continue  # Skip to the next URL
    except Exception as e:
        print(f"Unexpected error while processing {item.title}: {str(e)}")
        continue  # Skip any other unexpected errors

%store research_context

Retrieving research context for the resource: Qantas' brand health plummets; Virgin Australia sees gains in consumer sentiment
Retrieving research context for the resource: Domestic airline competition in Australia | May 2024 report
Retrieving research context for the resource: Insights on Australian travel behaviours
Stored 'research_context' (list)


/Users/sanandh/Desktop/Projects/Deep-Research-Python/venv/lib/python3.12/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/research_context requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


In [10]:
class ContextEval(BaseModel):
    further_research: bool
    probing_queries: Optional[SearchQueries]

def evaluate_context(prompt: str, objective: str, context: list[str], width: int) -> tuple[bool, Optional[list[str]]]:
    """Evaluates whether further research is needed based on the current context."""
    research_status: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content': f"""
            You are an expert analyst. You are provided with information regarding the user's topic of research, which is: {prompt}.
            The objective of this research is: {objective}.
            The research context we have until now is here: {context}.
            Based on the given information, confirm if further research is required.
            However, if you think more information is required for a comprehensive report, generate {width} more probing questions
            that go deeper into the topic, but make sure the queries are specific and precise to the current context as they will be used
            to fetch Google search results, which in turn will be used for further research.
            """
        }
    ], format=ContextEval.model_json_schema())
    
    context_eval = ContextEval.model_validate_json(research_status.message.content)
    return context_eval.further_research, context_eval.probing_queries

In [11]:
def research_loop(prompt: str, objective: str, context: list[str], width: int) -> list[str]:
    """Manages the iterative research process to refine the research context."""
    max_iterations = 2  # Prevent infinite loops
    iteration_count = 0
    current_context = context.copy()
    
    while iteration_count < max_iterations:
        further_research, probing_queries = evaluate_context(prompt, objective, current_context, width)
        
        if not further_research:
            break
        
        iteration_count += 1
        new_search_results = []
        
        if probing_queries:  
            for q in probing_queries.queries:  
                new_search_results.extend(fetch_serp(q))
                
            current_evaluated_queries, refs = evaluate_relevance(prompt, objective, width, new_search_results)
            
            for _ in current_evaluated_queries.r_queries:
                print(f"Retrieving research context for the resource: {_.title}")
                try:
                    new_context = retrieve_context(_.link)
                    references.extend(_)
                    current_context.append(new_context.get("markdown", ""))
                except HTTPError as e:
                    print(f"Failed to retrieve content from {_.title}: {str(e)}")
                    references.remove(_)
                    print(f"Removed '{_}' from references")
                    continue
                except Exception as e:
                    print(f"Unexpected error while processing {_.title}: {str(e)}")
                    continue
    
    if iteration_count >= max_iterations:
        print("Max iterations reached. Proceeding with the available research context.")
    
    return current_context

In [12]:
def generate_report(prompt:str, objective:str, width: int, sources: list[str]) -> str:
    final_context = research_loop(prompt, objective, research_context, width)
    final_report: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content':f"""You are an expert analyst. Your job is to use the provided research prompt, objective and context to generate a clear and comprehensive report.
            Research Title: {prompt}
            Research Objective: {objective}
            Research Context: {final_context}
            With this information, generate a detailed report as instructed. Be sure to include all sources, persons, objects etc in the report. You MUST include a references section and appropriately add citations, the references are here: {sources}. The report must be in markdown format. You are allowed to take liberties with section titles and in-report formatting. Ensure that the report itself adheres to the user's requirements and does not deviate away from the research's goals. You MUST remain aligned to the research title and objective, even if the context provided contains information that may cover aspects other than the ones mentioned by the user. Use the research title provided by the user for the report's title and follow the instructions as mentioned above."""
        }
    ])
    print(final_context)
    return final_report.message.content

In [13]:
report = generate_report(user_prompt,research_objective, research_width, references)

["Informa\n\nThis site is part of the Informa Markets Division of Informa PLC\n\n- [Informa PLC](https://informa.com/)\n- [About us](https://informa.com/about-us/)\n- [Investor relations](https://informa.com/investors/)\n- [Talent](https://informa.com/talent/)\n\nThis site is operated by a business or businesses owned by Informa PLC and all copyright resides with them. Informa PLC's registered office is 5 Howick Place, London SW1P 1WG. Registered in England and Wales. Number 8860726.\n\n[Skip to content](https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/#content)\n\n[![logo-header](https://www.corporatetravelcommunity.com/wp-content/uploads/elementor/thumbs/logo-header-qlfemu7qrdd279ngj81hhr9hdun0ycnrudfg53mv4i.png)](https://www.corporatetravelcommunity.com/)\n\nSearch\n\nSearch\n\n\\> Qantas’ brand health plummets; Virgin Australia sees gains in consumer sentiment\n\n# Qantas’ brand health plummets; Virgin Australia

In [14]:
display(Markdown(report))

**Marketing Strategies for Travel Agencies in Australia: Leveraging Behavioral Principles to Boost Bookings**
===========================================================

**Executive Summary**
-------------------

The travel industry in Australia is highly competitive, with numerous domestic airlines vying for market share. To gain a competitive edge, travel agencies must adopt effective marketing strategies that cater to the evolving needs of Australian travelers. This report explores the application of behavioral principles in marketing, focusing on social proof and risk aversion as key drivers of consumer decision-making.

**Introduction**
---------------

The Australian travel industry is characterized by intense competition among domestic airlines (ACCC, 2024). To stay ahead of the curve, travel agencies must develop innovative marketing strategies that appeal to the changing preferences of Australian travelers. This report delves into the application of behavioral principles in marketing, specifically social proof and risk aversion, to enhance bookings.

**Behavioral Principles in Marketing**
------------------------------------

Research has shown that behavioral principles play a significant role in shaping consumer behavior (Google/The Behavioural Architects, 2023). Two key principles – social proof and risk aversion – were identified as top performers for domestic flights (Social Proof and Risk Aversion) and international flights (Social Proof and Category Heuristics).

### Social Proof

*   54% of Australian travelers rely on other sources to verify the accuracy of travel-related information (Google/Ipsos, 2023).
*   Travelers are more likely to book with airlines that have a strong online presence and positive reviews (Qantas' brand health plummets; Virgin Australia sees gains in consumer sentiment, n.d.).

### Risk Aversion

*   Consumers exhibit risk aversion when booking travel arrangements, seeking assurance of the best possible outcome (Google/Ipsos, 2023).
*   Travel agencies can leverage social proof and positive reviews to mitigate perceived risks.

**Marketing Strategies**
-------------------------

To capitalize on behavioral principles, travel agencies can implement the following strategies:

1.  **Influencer Marketing**: Partner with social media influencers or travel bloggers to promote destinations and airlines.
2.  **User-Generated Content (UGC) Campaigns**: Encourage customers to share their experiences through UGC campaigns, amplifying social proof.
3.  **Risk Management Strategies**: Develop policies that address consumer concerns, such as flexible cancellation and refund policies.

**Implementation Roadmap**
---------------------------

1.  Conduct thorough market analysis to identify target audience preferences and behaviors.
2.  Develop tailored marketing campaigns incorporating behavioral principles (social proof and risk aversion).
3.  Monitor campaign performance, adjusting strategies as needed.

**Conclusion**
----------

The Australian travel industry is poised for growth with the implementation of data-driven marketing strategies that cater to consumer needs. By leveraging social proof and risk aversion, travel agencies can increase bookings and enhance their market share.

**References**
--------------

ACCC (2024). Domestic airline competition in Australia | May 2024 report. Retrieved from <https://www.accc.gov.au/system/files/domestic-airline-competition-in-australia-may-2024-report.pdf>

Google/Ipsos (2023). Travel Attitudes, Demand Research 2023, Australia.

Google/The Behavioural Architects (2023). [Top performing behavioural principles for domestic and international flights](https://www.thinkwithgoogle.com/intl/en-apac/insights/consumer-insights/travel-and-tourism/marketing-strategies-for-travel-agencies-in-australia-leveraging-behavioral-principles-to-boost-bookings/).

Qantas' brand health plummets; Virgin Australia sees gains in consumer sentiment (n.d.). Retrieved from <https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/>

Think with Google (2023). Insights on Australian travel behaviours. Retrieved from <https://www.thinkwithgoogle.com/intl/en-apac/consumer-insights/consumer-trends/insights-aunz-traveler-behaviors/>

In [15]:
references

[ResultItem(title="Qantas' brand health plummets; Virgin Australia sees gains in consumer sentiment", link='https://www.corporatetravelcommunity.com/qantas-brand-health-plummets-virgin-australia-sees-gains-in-consumer-sentiment/'),
 ResultItem(title='Domestic airline competition in Australia | May 2024 report', link='https://www.accc.gov.au/system/files/domestic-airline-competition-in-australia-may-2024-report.pdf'),
 ResultItem(title='Insights on Australian travel behaviours', link='https://www.thinkwithgoogle.com/intl/en-apac/consumer-insights/consumer-trends/insights-aunz-traveler-behaviors/')]